<a href="https://colab.research.google.com/github/tak595/Data-Minning-Midterm/blob/main/For_Midterm_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
#Pandas is to store and make functional the data we're going to use
import pandas as pd
#Sklearn is to test whether our method, Standard deviation, is working for finding the best feature(s)
from sklearn import linear_model as lm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


#Read in data
#taxData = pd.read_csv('/content/sales-and-use-tax.csv')
taxData = pd.read_csv('https://data.burlingtonvt.gov/explore/dataset/sales-and-use-tax/download/?format=csv&timezone=US/Eastern&lang=en&use_labels_for_header=true&csv_separator=%2C')
#popData = pd.read_csv('/content/HS-STAT-Population-of-Vermont-towns-1930-2019.csv')
popData = pd.read_excel('https://www.healthvermont.gov/sites/default/files/documents/xls/HS-STAT-Population-of-Vermont-towns-1930-2019.xls', skiprows=4)

#dropped an ID column
popData = popData.drop('CTC', axis=1)

#Data Engineering step(s)
#Reshaped the data to get ready for merging with the tax data
popData2 = popData.melt(id_vars=['NAME'], var_name="Calendar Year", value_name="Population")

#Changing data type(dtype) for merging
popData2 = popData2.astype({'Calendar Year' : 'int64'})

#Merging datasets into one dataset
DATA = taxData.merge(popData2, left_on=["Town", "Calendar Year"], right_on=["NAME", "Calendar Year"], how='inner')

X_train, X_test, Y_train, Y_test = train_test_split(DATA['Retail'], DATA['Gross'], test_size=.25)

#Feature we want to predict
TargetFeature = "Gross"
#Separate Features and Target(s)
DATAFeatures = DATA.drop(TargetFeature, axis=1)

#Data Mining Steps - Using standard deviation
MostImportantFeature = DATAFeatures.std().sort_values(ascending=False)[0:1]
LeastImportantFeature = DATAFeatures.std().sort_values()[0:1]

#Final Data Engineering Steps
#Find most important feature, by our Data mining methodology
FeatureName = MostImportantFeature.index[0]


#Testing our choice from Data Mining Results
#Setting up input arrays for test

xb = np.array(X_train)
xb = xb.reshape(-1,1)
yb = np.array(Y_train)
yb = yb.reshape(-1,1)

xb_test = np.array(X_test)
xb_test = xb.reshape(-1,1)
yb_test = np.array(Y_test)
yb_test = yb.reshape(-1,1)

bestModel = lm.LinearRegression().fit(xb,yb)
BMscore = bestModel.score(xb_test, yb_test)

#if the regression score is above threshold use the chosen feature
BestDataModel = pd.DataFrame(DATA[['Gross', 'Retail']])

#multivariate example
xtm = DATA[['Population', 'Gross Count', 'Retail Count', 'Use Count']]
#xt = xt.reshape(-1,1)
ytm = np.array(DATA[TargetFeature])
ytm = ytm.reshape(-1,1)

testModelm = lm.LinearRegression().fit(xtm,ytm)
Testscorem = testModelm.score(xtm,ytm)

#Showing the bad case from Data Mining Results
xw = np.array(DATA['Calendar Year'])
xw = xw.reshape(-1,1)
yw = np.array(DATA['Gross'])
yw = yw.reshape(-1,1)

xwtest = np.array(xw)
xwtest = xw.reshape(-1,1)
ywtest = np.array(yw)
ywtest = yw.reshape(-1,1)

worstModel = lm.LinearRegression().fit(xw,yw)
WMscore = worstModel.score(xw,yw)


print(DATA.std().sort_values(ascending=False))
print('\nThe Most Important feature, ', MostImportantFeature.index[0],', has an linear regression accuracy of:  ', BMscore, sep="")
print('\nThe parameters for the best model are :\n m = ', bestModel.coef_, '\n b = ', bestModel.intercept_)
print('\nUsing multiple input features and avoiding other tax revenue components, gives a linear regression accuracy of:  ', Testscorem, sep="")
print('\nThe parameters for the best multivariate model are :\n m = ', testModelm.coef_, '\n b = ', testModelm.intercept_)
print('\nThe Least Important feature, ', LeastImportantFeature.index[0],', has an linear regression accuracy of:  ', WMscore, sep="")
print('\nThe best data model is:  \n', BestDataModel.head(), '\n')

yb_pred = bestModel.predict(xb_test)
yw_pred = worstModel.predict(xw)
#yt_pred = testModel.predict(xt)

plt.figure(0)
plt.scatter(xb_test, yb_test)
plt.plot(xb_test, yb_pred, color='red')

print(len(DATA['Retail']))
print(len(X_train))

plt.figure(1)
plt.scatter(xw, yw)
plt.plot(xw, yw_pred, color='red')
